In [35]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
df = pd.read_csv('csv/movies.csv')

In [36]:
keywords = df['keywords'].dropna()

In [30]:
# Для размышлений
from difflib import SequenceMatcher

space = tuple(filter(lambda x: x.lower().startswith('space'), keywords))

In [33]:
[(SequenceMatcher(None, space[0], i).ratio(), i) for i in space]

[(1.0, 'space marine'),
 (0.5833333333333334, 'space colony'),
 (0.6086956521739131, 'space opera'),
 (0.5882352941176471, 'space'),
 (0.5714285714285714, 'spaceship'),
 (0.5714285714285714, 'spaceport'),
 (0.7272727272727273, 'space race'),
 (0.72, 'space mission'),
 (0.6666666666666666, 'space pirate'),
 (0.8, 'space carrier'),
 (0.5333333333333333, 'space based weapon'),
 (0.72, 'space station'),
 (0.8, 'spaceman'),
 (0.6153846153846154, 'space invasion'),
 (0.5454545454545454, 'spacecraft'),
 (0.7619047619047619, 'space war'),
 (0.6363636363636364, 'space suit'),
 (0.6666666666666666, 'space travel'),
 (0.6666666666666666, 'space battle'),
 (0.5806451612903226, 'spacecraft accident'),
 (0.6666666666666666, 'space centre'),
 (0.4666666666666667, 'spacecraft officer')]

In [37]:
tokenizer = Tokenizer(num_words=8000,
                      lower=True,
                      split=';')

In [39]:
tokenizer.fit_on_texts(keywords)

In [65]:
len(keywords[0].split(';'))

21

In [ ]:
def to_binary(words_indexes, num_words=8000):
     binary_words = np.zeros(num_words)
     binary_words[words_indexes] = 1
     return binary_words

In [64]:
words = list(filter(bool, [x[0] if x else 0 for x in tokenizer.texts_to_sequences(keywords[0].split(';'))]))
print(len(words))
binary_words = to_binary(words)
binary_words

21


array([0., 0., 0., ..., 0., 0., 0.])

In [66]:
(binary_words == 1).sum(), (binary_words == 0).sum()

(21, 7979)

In [67]:
import pickle

with open('tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
from tokenizer import CustomTokenizer
my_tokenizer = CustomTokenizer()

TypeError: 'int' object is not subscriptable

In [2]:
my_tokenizer.tokenizer.index_word

NameError: name 'my_tokenizer' is not defined